In [21]:
%matplotlib notebook

import pyaudio
from pyaudio import PyAudio, paFloat32
from time import sleep
from scipy.io.wavfile import write
from numpy import linspace,sin,pi,int16,concatenate,ndarray,array,int64,int32,float32,zeros,random,arange,float64
from pylab import plot,show,axis

import matplotlib.pyplot as plt
import pandas as pd
import os
import random as rand
import cmath
from os.path import isfile, join 

In [15]:
fileList = os.listdir('gg/')
noteDir = os.listdir('scl/')

def isDataFile(filename):
    return filename.find('.dat')>0

dataFiles = filter(isDataFile,fileList)

def isNoteFile(filename):
    return filename.find('.txt')>0

noteFiles = filter(isNoteFile,noteDir) #Pulls all .txt files containing the various scales, same as data files func

In [ ]:
h = 50
h/5 

In [16]:
scales = {} #Initialize overall scale dictionary
all_grains = [] #Initialize overall list of grain snapshots

print 'Building note list...'
for scale in noteFiles:
    notes = open('scl/' + scale,'r')
    note_list = []
    note_file = notes.readlines()
    for line in note_file:
        line = line.split("\t",3)
        line[2] = line[2].strip('\n')
        note_list.append(line)
    scales[scale.strip('.txt')] = note_list

print 'Note list built.'

print 'Building grain list...'

for data in dataFiles:
    grain_list = []
    grains = open("gg/" + data, "r")
    grain_file = grains.readlines()
    for line in grain_file:
        line = line.split("\t",3)
        line[2] = line[2].strip('\n')
        grain_list.append(line)
    all_grains.append(grain_list)

print 'Grain list built.'

Building note list...
Note list built.
Building grain list...
Grain list built.


In [ ]:
len(all_grains[0])

In [ ]:
pd.read_csv('gg/'+dataFiles[0],sep='\t',header=None,names=('grain-ID','area','sides'))

In [17]:
# tone synthesis
def note(freq, len, amp=1, rate=44100):
 t = linspace(0,len,len*rate)
 data = sin(2*pi*freq*t)*amp
 return data.astype(int16) # two byte integers

In [20]:
################################### Constants ##################################

fs            = 44100   # Hz
threshold     = 0.8     # absolute gain
delay         = 40      # samples
signal_length = 1       # second
release_coeff = 0.9999  # release time factor
attack_coeff  = 0.9     # attack time factor
dtype         = float32 # default data type
block_length  = 1024    # samples

#################### Generate quiet-loud-quiet noise signal ####################

signal = array(random.rand(fs*signal_length)*2-1, dtype=dtype)
signal[:signal_length*fs/3] *= 0.1
signal[signal_length*fs*2/3:] *= 0.1

############################# Implementation of Limiter ########################

class Limiter:
    def __init__(self, attack_coeff, release_coeff, delay, dtype=float32):
        self.delay_index = 0
        self.envelope = 0
        self.gain = 1
        self.delay = delay
        self.delay_line = zeros(delay, dtype=dtype)
        self.release_coeff = release_coeff
        self.attack_coeff = attack_coeff

    def limit(self, signal, threshold):
        for i in arange(len(signal)):
            self.delay_line[self.delay_index] = signal[i]
            self.delay_index = (self.delay_index + 1) % self.delay

            # calculate an envelope of the signal
            self.envelope *= self.release_coeff
            self.envelope  = max(abs(signal[i]), self.envelope)

            # have self.gain go towards a desired limiter gain
            if self.envelope > threshold:
                target_gain = (1+threshold-self.envelope)
            else:
                target_gain = 1.0
            self.gain = ( self.gain*self.attack_coeff +
                          target_gain*(1-self.attack_coeff) )

            # limit the delayed signal
            signal[i] = self.delay_line[self.delay_index] * self.gain

################################# Play the Audio ###############################

original_signal = array(signal, copy=True, dtype=dtype)

limiter = Limiter(attack_coeff, release_coeff, delay, dtype)

def callback(in_data, frame_count, time_info, flag):
    if flag:
        print("Playback Error: %i" % flag)
    played_frames = callback.counter
    callback.counter += frame_count
    limiter.limit(signal[played_frames:callback.counter], threshold)
    return signal[played_frames:callback.counter], pyaudio.paContinue

callback.counter = 0

pa = PyAudio()

stream = pa.open(format = paFloat32,
                 channels = 1,
                 rate = fs,
                 frames_per_buffer = block_length,
                 output = True
                 stream_callback = callback)

while stream.is_active():
    sleep(0.1)

stream.close()
pa.terminate()

############################## Plot results ####################################

plt.figure()
plt.plot(original_signal, color='grey', label='original signal')
plt.plot(signal, color='black', label='limited signal')
plt.legend()
plt.show(block=True)

SyntaxError: invalid syntax (<ipython-input-20-5d660477b3af>, line 73)

In [22]:
# A tone, 2 seconds, 44100 samples per second (PH note)

tone_total = []
notefile = 0
i = 0
for data in all_grains:
    i += 1
    scale_selection = rand.randint(0,3)
    tones = []
    for grain in data:
        for tone in scales[noteFiles[notefile].strip('.txt')]:
            if grain[2] == tone[2]:
                output_note = note(float(tone[1]),float(2)/len(data),amp=100000*float(grain[1]))
                tone_total.append(output_note)
    if i%5 == 0:
        notefile += 1
    if i/10 == 2:
        notefile = 0
        i = 0
    #tone_total.append(sum(tones))

chords = []
chordfile = 0
w = 0
for data in all_grains:
    w += 1
    scale_selection = rand.randint(0,3)
    tones = []
    for grain in data:
        for tone in scales[noteFiles[chordfile].strip('.txt')]:
            if grain[2] == tone[2]:
                output_note = note(float(tone[1]),2,amp=1000*float(grain[1]))
                tones.append(output_note)
    chords.append(sum(tones))
    if w%5 == 0:
        chordfile += 1
    if w/10 == 2:
        chordfile = 0
        w = 0
    
print 'Finished building notes'




#concatenate((tone_total))

Finished building notes


In [ ]:
print len(tone_total)
print len(tone_total[0])
print tone_total[-1]
all_notes = []
for note in tone_total:
    if len(all_notes) == 0:
        all_notes.append(int32(note))
    else:
        all_notes.extend(note)
print len(all_notes)
print all_notes[-1]

In [23]:
write('test_noteProgression.wav',44100,concatenate((tone_total))) # writing the sound to a file
write('test_chords.wav',44100,concatenate((chords))) # writing the sound to a file
print len(tone_total[:])
print len(chords[:])
#plot(linspace(0,2,2*44100),tone_total)
#axis([0,0.4,15000,-15000])
#show()

5000
100
